In [79]:
#helper libraries
import numpy as np
from numpy.random import rand
import math
import scipy.optimize
import matplotlib.pyplot as plt

#qiskit functionalities
from qiskit import Aer, execute, QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.tools.visualization import plot_histogram

#noise model stuff
from qiskit.providers.aer.noise import NoiseModel, kraus_error, pauli_error
    
#operator info
from qiskit.quantum_info import PauliList, Pauli, Kraus
from qiskit.circuit.library import XGate

import PauliExperimentHelperFunctions as pe

In [82]:
n = 1 #Number of qubits
F = PauliList(['I', 'X', 'Y', 'Z']) #list of paulis
T = PauliList(['X', 'Y', 'Z']) #paulis considered in the model
coeffs = [.01, .05, .0085] #model coefficients

#Kraus error channel
noise_model = pe.build_noise_model(F, T, coeffs, n)

backend = Aer.get_backend('qasm_simulator')
gate = XGate() #the gate to use for noise profiling
folds = 40 #number of folds to fit noise
shots = 4096 #number of shots for expectation values

#generate the circuits with increasing numbers of folds
circuits = pe.generate_circuits('X', gate, folds)
#extract expectation values at each circuit depth
xfidelities = pe.fidelity_experiment(circuits, backend, noise_model,shots)

#repeat for <Y>
circuits = pe.generate_circuits('Y', gate, folds)
yfidelities = pe.fidelity_experiment(circuits, backend, noise_model,shots)

#repeat for <Z>
circuits = pe.generate_circuits('Z', gate, folds)
zfidelities = pe.fidelity_experiment(circuits, backend, noise_model,shots)

measured_fidelities = pe.learn_fidelities(xfidelities, yfidelities, zfidelities)
#get the coefficients from the fit
measured_coeffs = pe.learn_model_coefficients(measured_fidelities, F, T)

def unzip(zippedlist):
    return [a for (x,a) in zippedlist]

def manhattannorm(a, b):
    return np.sum(np.abs(np.subtract(a,b)))

print("Manhattan distance between measured/ideal fidelity vectors: ", "%.5f" %manhattannorm(unzip(pe.get_ideal_fidelities(F, T, coeffs)), unzip(measured_fidelities)))
print("Manhattan distance between measured/ideal model coefficients: ", "%.5f" %manhattannorm(unzip(measured_coeffs), coeffs))

Manhattan distance between measured/ideal fidelity vectors:  0.00457
Manhattan distance between measured/ideal model coefficients:  0.00263


In [93]:
samples = 1000 #number of circuits toi sample

overhead = np.exp(2*np.sum(coeffs)) #Compute the sampling overhead

ideal_circuit = QuantumCircuit(qr, cr)
ideal_circuit.x(0)
ideal_circuit.measure(0,0); #expectation value 

In [17]:
job = execute(ideal_circuit, backend, noise_model = kraus_noise_model, basis_gates = kraus_basis_gates, shots=10000)
unmitigated_value = expectation(job.result().get_counts())
print(unmitigated_value)

-0.8862


In [73]:
ideal_circuit = QuantumCircuit(qr, cr)
ideal_circuit.x(0)

#Follow the procedure for sampling gates
def uniform_noise_scaling(lmbda, samples, ideal_circuit, measured_coeffs, backend, noise_model):
    circuits = []
    sgns = []

    adjusted_omegas = .5*(np.exp(np.multiply(-2*(1-lmbda), measured_coeffs))+1)
    
    for i in range(samples):
        m = 0 #m keeps track of the sign, with paulis carrying a negative sign
        qc = ideal_circuit.copy()
        op = Pauli('I')
        for (omega, Pk) in zip(omegas, pauli_list):
            #with probability 1-\omega_k, sample the Pauli gate and compose into operator
            if rand() < 1-omega:
                m+=1
                op = op.compose(Pk) #Wow I spent so long on this silly line
        qc.barrier()
        qc.append(op.to_instruction(),[0])
        qc.barrier()
        qc.measure(0,0)
        circuits.append(qc)
        sgns.append((-1)**m)
    
    
    job = execute(circuits, backend, noise_model = noise_model, basis_gates = noise_model.basis_gates, shots = 1)
    
    overhead = math.exp(2*(1-lmbda)*np.sum(measured_coeffs))
    
    total = 0
    for (count,sgn) in zip(job.result().get_counts(),sgns):
        total += expectation(count)*sgn*overhead

    mitigated_value = total/samples
    
    return mitigated_value